In [2]:
import os
import datetime
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
import shutil

In [2]:
rootfolder = "G:\\"
#folder = rootfolder+"CraneData\\"
folder = "E:\\Documents\\Dan\\Port Work\\OneDrive\\OneDrive - The Port of Virginia\\Shared with Everyone\\Crane Data\\"
SensorPositonFile = rootfolder + 'SensorStatsSmall.csv'
OutputVectors = np.genfromtxt(open(SensorPositonFile,'r'), delimiter=',',skip_header=1,dtype=int, missing_values=0)

In [3]:
def IndexFile(Filename):
    
    try:
        #Parsing the file name.  Assuming it is in the standard format
        sSensor = Filename[23]
        sDate = datetime.datetime.strptime('20'+Filename[10:21],"%Y%m%d-%H%M")

        i=0
        j = 0
        #loops through the known sensor movements, and creates a filter mask
        for spf in OutputVectors:
            
            startDate = datetime.datetime.strptime(str(spf[0])+str(spf[1]).zfill(2)+str(spf[2]).zfill(2)
                +str(spf[3]).zfill(2)+str(spf[4]).zfill(2),"%Y%m%d%H%M")
            #datetime.date(int(spf[0]), int(spf[1]), int(spf[2])) + datetime.timedelta(hours=spf[3]) + datetime.timedelta(minutes=spf[4])
            endDate = datetime.datetime.strptime(str(spf[5])+str(spf[6]).zfill(2)+str(spf[7]).zfill(2)
                +str(spf[8]).zfill(2)+str(spf[9]).zfill(2),"%Y%m%d%H%M")
            #datetime.date(int(spf[5]), int(spf[6]), int(spf[7])) + datetime.timedelta(hours=spf[8]) + datetime.timedelta(minutes=spf[9])
            
            if sDate >= startDate and sDate <= endDate and int(spf[10]) == int(sSensor):
                i+=1
                result = j
            else:
                pass
            j += 1
            
        if i != 1: 
            print('error ', i, Filename)
            result = np.shape(OutputVectors)[0]
    except:
        result = 'fail'

 
    return result

In [9]:
def CheckArrayOfFiles(Files):

    Results = np.zeros(np.shape(OutputVectors)[0]+2)

    for file in Files:
        if file [-3:] != 'csv': 
            Results[np.shape(OutputVectors)[0]+1] +=1
        else:
            ind = IndexFile(file)
            Results[ind] += 1

    return Results

In [5]:
Groups = multiprocessing.cpu_count() -2

In [10]:
AllFiles = os.listdir(folder)

In [11]:
AllFiles = [AllFiles[x:x+Groups] for x in range(0, len(AllFiles), Groups)]

In [12]:
Results = Parallel(n_jobs=Groups)(delayed(CheckArrayOfFiles)(ListFiles) for ListFiles in AllFiles)

In [13]:
np.matrix(Results)
Final = np.sum(Results, axis = 0)

In [14]:
Final

array([1.7200e+02, 2.0000e+00, 4.1600e+02, 4.9100e+02, 3.0000e+02,
       0.0000e+00, 6.4700e+02, 4.3900e+02, 4.3400e+02, 3.7500e+02,
       2.8500e+02, 4.6700e+02, 2.7800e+02, 3.3000e+02, 4.6600e+02,
       3.3300e+02, 1.2240e+03, 1.0590e+03, 7.9900e+02, 2.8400e+02,
       9.3600e+02, 3.0100e+02, 5.0500e+02, 6.1600e+02, 7.2500e+02,
       5.1200e+02, 7.9800e+02, 1.4350e+03, 2.1130e+03, 8.1000e+02,
       9.9500e+02, 8.8200e+02, 8.4900e+02, 1.6100e+03, 9.4100e+02,
       1.2300e+02, 1.9540e+03, 9.8900e+02, 1.6400e+03, 1.5860e+03,
       1.4960e+03, 5.8870e+03, 5.6610e+03, 2.2000e+01, 1.7180e+03,
       0.0000e+00, 9.0000e+00, 3.0930e+03, 1.2370e+03, 9.3200e+02,
       9.5100e+02, 1.2820e+03, 2.0180e+03, 2.2110e+03, 1.5710e+03,
       2.1520e+03, 1.3205e+04, 1.6283e+04, 2.1890e+03, 1.5650e+03,
       9.4490e+03, 6.8530e+03, 1.6808e+04, 1.1513e+04, 0.0000e+00,
       2.0880e+04, 7.1750e+03, 1.1000e+01, 1.5660e+03, 1.7490e+03,
       2.1620e+03, 0.0000e+00, 7.5820e+03, 6.6300e+02, 0.0000e

In [ ]:
def CheckValidFileName(FileName):
    test = IndexFile(FileName)
    if test == 'fail':
        result = True
    else:
        result = False
    return result

In [ ]:
def CheckListofFileNames(Files):
    BadGuys = []
    for file in Files:
        Check = CheckValidFileName(file)
        if Check == True:
            BadGuys.append(file)

    return BadGuys

In [ ]:
BadGuys = Parallel(n_jobs=Groups)(delayed(CheckListofFileNames)(ListFiles) for ListFiles in AllFiles)


In [ ]:
j=[]
for r in BadGuys:
    if r ==[]:
        pass
    else:
        for t in r:
            if t ==[]: pass
            else:
                j.append(r)

In [ ]:
j

In [ ]:
a=0
b=0
c=0
folder = "C:\\Users\\Dan\\Desktop\\Temp\\"
FLists = os.listdir(folder)
for FLists in j:
    for Filename in FLists:
        #print(Filename, Filename[-3:], Filename[-6:-5], Filename[3:7])
        #if Filename[0:5] == 'Accel':
        newName = '60kPoints' + str(Filename[8:]) 
        os.rename(os.path.join(folder,Filename), 
            os.path.join(folder,newName))
        a+=1
        #if Filename [-3:] == 'csv' and Filename[-6:-5] != 's':
        #    newName = folder+Filename[:-4]+ '-s1a.csv'
        #    os.rename(folder+Filename, newName)
        #    b+=1
        #if Filename[3:7] == 'Gyro':
        #    shutil.move(folder+Filename, folder + 'Gyro\\'+Filename)
        #    c+=1

print(a,b,c)

In [ ]:
import os

dir = ("C:\\test")

# check that the folder exists
if os.path.isdir(dir):
    print('folder exists')
else:
    print('folder does not exist')
    exit()

for f in os.listdir(dir):

    a = os.path.join(dir, f)
    f2 = '60kPoints' + f[8:]
    b = os.path.join(dir, f2).replace('_', '').replace(' ', '').lower()
    os.rename(a, b)
    print('file names updated')



In [19]:
import os
import datetime
import shutil

Sensor = 2
Groups = 2
startDate = datetime.datetime.strptime(str(202205261537),"%Y%m%d%H%M")
            #datetime.date(int(spf[0]), int(spf[1]), int(spf[2])) + datetime.timedelta(hours=spf[3]) + datetime.timedelta(minutes=spf[4])
endDate = datetime.datetime.strptime(str(202206021200),"%Y%m%d%H%M")
folder = "E:\\Documents\\Dan\\Port Work\\OneDrive\\OneDrive - The Port of Virginia\\Shared with Everyone\\Crane Data\\"
FLists = os.listdir(folder)
NewFolder = "C:\\Users\\Dan\\OneDrive\\For F8F\\Crane Moves\\RMG 4w\\"

def CopyFile(Filename):
    s=0
    
    if Filename[-3:] == 'csv':
        sSensor = int(Filename[23])
        sDate = datetime.datetime.strptime('20'+Filename[10:21],"%Y%m%d-%H%M")
        
        if sDate >= startDate and sDate <= endDate and Sensor == int(sSensor):
            shutil.copyfile(folder+Filename, NewFolder+Filename)
            #print('Yes', Filename, sSensor)
            s=1
    return s
z=0

for file in FLists:
    z += CopyFile(file)


In [16]:
h = datetime.datetime.strptime('20'+FLists[0][10:21],"%Y%m%d-%H%M")
print(h , startDate, endDate)    

2021-07-09 07:15:00 2022-05-26 15:37:00 2022-06-02 12:00:00
